# Homework 1

_**[Power Systems Optimization](https://github.com/east-winds/power-systems-optimization)**_

_by Jesse D. Jenkins and Michael R. Davidson (last updated: September 14, 2022)_

This Notebook will walk you through defining a simple transport flow model and then ask you to interact with the solutions and modify to model to add additional constraints...

## Setting up the model

### Load packages

In [1]:
using JuMP
using HiGHS
using DataFrames
using CSV

### Define sets

We will define two sets, both as arrays of strings

***Production plants, $P$***

In [2]:
P=["trenton", "newark"] # production plants

2-element Array{String,1}:
 "trenton"
 "newark"

***Markets for products, $M$***

In [3]:
M=["newyork", "princeton", "philadelphia"] # markets for products

3-element Array{String,1}:
 "newyork"
 "princeton"
 "philadelphia"

Note that sets can also be defined over intervals (as in `i=1:10`) or numerical vectors (as in `x=[2, 4, 5, 11]`) 

### Define parameters

We'll make use of the defined sets as indexes for our parameters...

***Plant production capacities***

In [4]:
plants = DataFrame(plant=P, capacity=[350,650])

,plant,capacity
,String,Int64
1,trenton,350
2,newark,650


***Demand for products***

Stored in a [DataFrame](https://juliadata.github.io/DataFrames.jl/stable/)

In [5]:
markets = DataFrame(
    market=M, 
    demand=[325, 300, 275]
)

,market,demand
,String,Int64
1,newyork,325
2,princeton,300
3,philadelphia,275


A few different ways to index into our DataFrames to access parameters (all of the below are equivalent)

In [6]:
plants[plants.plant.=="newark",:capacity] # option 1

1-element Array{Int64,1}:
 650

In [7]:
plants[plants.plant.=="newark",:].capacity # option 2

1-element Array{Int64,1}:
 650

In [8]:
plants.capacity[plants.plant.=="newark"] # option 3

1-element Array{Int64,1}:
 650

In [9]:
plants[:,:capacity][plants.plant.=="newark"] # option 4

1-element Array{Int64,1}:
 650

Note that DataFrame indexing returns an Array by default, in this case, a 1-element Array of type Int64 (64-bit integer), as indicated by `Array{Int64,1}` above. 

To access the single Int64 value, append `[1]` to any of the above to reference the first (and only) element in this array. 

In [10]:
plants.capacity[plants.plant.=="newark"][1]

650

In [11]:
typeof(plants.capacity[plants.plant.=="newark"][1])

Int64

In [12]:
typeof(plants.capacity[plants.plant.=="newark"])

Array{Int64,1}

***Distance from plants to markets***

Stored in a JuMP [DenseAxisArray](https://jump.dev/JuMP.jl/v0.19/containers/) with data array and symbolic references across each of our sets (plants and markets), converted to Symbols for referencing

In [13]:
# two dimensional symbolic DenseAxisArray, with from/to distance pairs
distances = JuMP.Containers.DenseAxisArray(
    [2.5 0.5 1.5;
     0.5 1.5 3.5],
    Symbol.(P),
    Symbol.(M),
)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, [:trenton, :newark]
    Dimension 2, [:newyork, :princeton, :philadelphia]
And data, a 2×3 Array{Float64,2}:
 2.5  0.5  1.5
 0.5  1.5  3.5

A couple example references to our DenseAxisArray to access parameters...

In [14]:
distances[:trenton, :newyork] #example of distance references

2.5

In [15]:
distances[:newark, :newyork] #example of distance references

0.5

In [16]:
distances[Symbol(P[2]),Symbol(M[1])] # another way to find distance from newark to trenton

0.5

In [17]:
distances[Symbol("newark"), Symbol("newyork")] # and a third...

0.5

***Costs of transport***

In [18]:
freight_cost = 90 # Cost of freight shipment per unit of distance

90

### Create model
(and specify the HiGHS solver)

In [19]:
transport = Model(HiGHS.Optimizer);

### Define variables

***Quantities of product to transport from plant $p \in P$ to market $m \in M$***

In [20]:
@variable(transport, X[P,M] >= 0)

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, ["trenton", "newark"]
    Dimension 2, ["newyork", "princeton", "philadelphia"]
And data, a 2×3 Array{VariableRef,2}:
 X[trenton,newyork]  X[trenton,princeton]  X[trenton,philadelphia]
 X[newark,newyork]   X[newark,princeton]   X[newark,philadelphia]

Example reference to single quantity decision variable, the quantity shipped from Newark to Philadelphia:

In [21]:
X["newark","philadelphia"]

X[newark,philadelphia]

### Define constraints

***Supply capacity constraint***

In [22]:
@constraint(transport, cSupply[p in P], 
    sum(X[p,m] for m in M) 
    <= plants.capacity[plants.plant.==p][1])

1-dimensional DenseAxisArray{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1,...} with index sets:
    Dimension 1, ["trenton", "newark"]
And data, a 2-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1}:
 cSupply[trenton] : X[trenton,newyork] + X[trenton,princeton] + X[trenton,philadelphia] ≤ 350.0
 cSupply[newark] : X[newark,newyork] + X[newark,princeton] + X[newark,philadelphia] ≤ 650.0

***Demand balance constraint***

Ensure all demand is satisfied at each market

In [23]:
@constraint(transport, cDemand[m in M], 
    sum(X[p,m] for p in P) 
    >= markets.demand[markets.market.==m][1])

1-dimensional DenseAxisArray{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.GreaterThan{Float64}},ScalarShape},1,...} with index sets:
    Dimension 1, ["newyork", "princeton", "philadelphia"]
And data, a 3-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.GreaterThan{Float64}},ScalarShape},1}:
 cDemand[newyork] : X[trenton,newyork] + X[newark,newyork] ≥ 325.0
 cDemand[princeton] : X[trenton,princeton] + X[newark,princeton] ≥ 300.0
 cDemand[philadelphia] : X[trenton,philadelphia] + X[newark,philadelphia] ≥ 275.0

### Define objective function

Minimize total cost of transport to satisfy all demand.

First we'll define an expression for total cost of shipments...

In [24]:
@expression(transport, eCost, 
    sum(freight_cost*distances[Symbol(p),Symbol(m)]*X[p,m] 
        for p in P, m in M)
    )

225 X[trenton,newyork] + 45 X[trenton,princeton] + 135 X[trenton,philadelphia] + 45 X[newark,newyork] + 135 X[newark,princeton] + 315 X[newark,philadelphia]

Now we'll minimize this total cost

In [25]:
@objective(transport, Min, eCost)

225 X[trenton,newyork] + 45 X[trenton,princeton] + 135 X[trenton,philadelphia] + 45 X[newark,newyork] + 135 X[newark,princeton] + 315 X[newark,philadelphia]

## Interact with the model

**(a)** Now let's solve the model. In the blank cell below, enter the command for JuMP to solve a model and run the cell

In [26]:
optimize!(transport)


Presolving model
5 rows, 6 cols, 12 nonzeros
5 rows, 6 cols, 12 nonzeros
Presolve : Reductions: rows 5(-0); columns 6(-0); elements 12(-0)
Solving the presolved LP
Using EKK dual simplex solver - serial
LP  has all |entries|=1; max column count = 2 (limit 24); average column count = 2 (limit 6): So is a candidate for LiDSE
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 3(900) 0s
          4     8.5500000000e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 4
Objective value     :  8.5500000000e+04
HiGHS run time      :          0.01


**(b)** You've got a solution. Now query the objective function in the empty cell below and save it to a variable (name of your choice)

In [27]:
value(eCost)

85500.0

**(c)** Now query and save the optimal solution for X (the decisions about shipment quantities from plant to market) to an Array or DataFrame

In [28]:
value.(X)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, ["trenton", "newark"]
    Dimension 2, ["newyork", "princeton", "philadelphia"]
And data, a 2×3 Array{Float64,2}:
   0.0   75.0  275.0
 325.0  225.0    0.0

**(d)** Please interpret your results by writing an explanation in the markdown cell below. 

Which facility or facilities supplies the most demand in New York? Does this result make sense? Why?

Which facility or facilities supplies the most demand in Philadelphia? Does this result make sense? Why?

Which facility or facilities supplies the demand in Princeton? Does this result make sense? Why?

- The Newark facility supplies all the demand for NY. This is sensible as the Newark plant is much larger then the Trenton plant, and located physically closer to NYC then Trenton.
- For Princeton, Newark supplies more power, but Trenton also contributes a sizeable amount, as Trenton is much closer.
- For Philly, Trenton supplies all of the demand, as it's physically the closest.

In context of all three, there results make a lot of sense. Newark supplies the most demand in NYC, as it's closest, and sends the extra to Princeton. Trenton picks up the rest of Princeton and then all of Philidelphia. 

**(e)** A new market in New Brunswick appears, with a demand for 50 units. It is located 1.0 units away from both plants. Add this market to the model and solve again.

In [29]:
M=["newyork", "princeton", "philadelphia", "newbrunswick"] # markets for products
markets = DataFrame(
    market=M, 
    demand=[325, 300, 275, 50]
)
# two dimensional symbolic DenseAxisArray, with from/to distance pairs
distances = JuMP.Containers.DenseAxisArray(
    [2.5 0.5 1.5 1.0;
     0.5 1.5 3.5 1.0],
    Symbol.(P),
    Symbol.(M),
)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, [:trenton, :newark]
    Dimension 2, [:newyork, :princeton, :philadelphia, :newbrunswick]
And data, a 2×4 Array{Float64,2}:
 2.5  0.5  1.5  1.0
 0.5  1.5  3.5  1.0

In [32]:
transport = Model(HiGHS.Optimizer);
@variable(transport, X[P,M] >= 0)
@constraint(transport, cSupply[p in P], 
    sum(X[p,m] for m in M) 
    <= plants.capacity[plants.plant.==p][1])
@constraint(transport, cDemand[m in M], 
    sum(X[p,m] for p in P) 
    >= markets.demand[markets.market.==m][1])
@expression(transport, eCost, 
    sum(freight_cost*distances[Symbol(p),Symbol(m)]*X[p,m] 
        for p in P, m in M)
    )
@objective(transport, Min, eCost)

225 X[trenton,newyork] + 45 X[trenton,princeton] + 135 X[trenton,philadelphia] + 90 X[trenton,newbrunswick] + 45 X[newark,newyork] + 135 X[newark,princeton] + 315 X[newark,philadelphia] + 90 X[newark,newbrunswick]

In [33]:
optimize!(transport)


Presolving model
6 rows, 8 cols, 16 nonzeros
6 rows, 8 cols, 16 nonzeros
Presolve : Reductions: rows 6(-0); columns 8(-0); elements 16(-0)
Solving the presolved LP
Using EKK dual simplex solver - serial
LP  has all |entries|=1; max column count = 2 (limit 24); average column count = 2 (limit 6): So is a candidate for LiDSE
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 4(950) 0s
          5     9.0000000000e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 5
Objective value     :  9.0000000000e+04
HiGHS run time      :          0.01


**(f)** What is new optimal solution? 

In [34]:
value.(X)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, ["trenton", "newark"]
    Dimension 2, ["newyork", "princeton", "philadelphia", "newbrunswick"]
And data, a 2×4 Array{Float64,2}:
   0.0   75.0  275.0   0.0
 325.0  225.0    0.0  50.0

**(g)** Interpret this result in the markdown cell below. Which facility or facilities supplies the demand in New Brunswick? Does this result make sense? Why?

- Newark picks up the slack, as it was not running at capacity to start with before adding New Brunswick, unlike Trenton. 
- Given that New Brunswick is eqidistant, the model simply tacks on the extra demand to Newark. 


What happens if this is not the case?

Suppose the distance to Newark is now 5 times higher:

In [36]:
distances = JuMP.Containers.DenseAxisArray(
    [2.5 0.5 1.5 1.0;
     0.5 1.5 3.5 5.0], #  change the location of New Brunswick to be further from Newark.
    Symbol.(P),
    Symbol.(M),
)
transport = Model(HiGHS.Optimizer);
@variable(transport, X[P,M] >= 0)
@constraint(transport, cSupply[p in P], 
    sum(X[p,m] for m in M) 
    <= plants.capacity[plants.plant.==p][1])
@constraint(transport, cDemand[m in M], 
    sum(X[p,m] for p in P) 
    >= markets.demand[markets.market.==m][1])
@expression(transport, eCost, 
    sum(freight_cost*distances[Symbol(p),Symbol(m)]*X[p,m] 
        for p in P, m in M)
    )
@objective(transport, Min, eCost)
optimize!(transport)
value.(X)


Presolving model
6 rows, 8 cols, 16 nonzeros
6 rows, 8 cols, 16 nonzeros
Presolve : Reductions: rows 6(-0); columns 8(-0); elements 16(-0)
Solving the presolved LP
Using EKK dual simplex solver - serial
LP  has all |entries|=1; max column count = 2 (limit 24); average column count = 2 (limit 6): So is a candidate for LiDSE
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 4(950) 0s
          5     9.4500000000e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 5
Objective value     :  9.4500000000e+04
HiGHS run time      :          0.00


2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, ["trenton", "newark"]
    Dimension 2, ["newyork", "princeton", "philadelphia", "newbrunswick"]
And data, a 2×4 Array{Float64,2}:
   0.0   25.0  275.0  50.0
 325.0  275.0    0.0   0.0

In this case, the model shuffles production around so that the now much closer Trenton plant picks up the demand for New Brunswick.